# Chains with multiple inputs


In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

llm = ChatOpenAI()

prompt_template = PromptTemplate(input_variables=["input"], template="Tell me a joke about {input}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke(input="a parrot")

{'input': 'a parrot',
 'text': 'Why did the parrot wear a raincoat?\n\nBecause he wanted to be polyunsaturated!'}

In [3]:
prompt_template = PromptTemplate(input_variables=["input", "language"], template="Tell me a joke about {input} in {language}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke({"input": "a parrot", "language": "german"})

{'input': 'a parrot',
 'language': 'german',
 'text': 'Warum hat der Papagei den Bus verpasst?\n\nWeil er noch schnell einen Vogel hatte!'}